In [5]:
import os
%pwd
os.chdir("./end-to-end--ml-pipeline")
%pwd

'/home/user/shahabas/MLops/end-to-end--ml-pipeline'

In [8]:
import pandas as pd

In [10]:
data_path="artifacts/data_ingestion/winequality-red.csv"
wines=pd.read_csv(data_path)

In [11]:
from dataclasses import dataclass
from pathlib import Path

In [12]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir : Path
    unzip_data_dir : Path
    STATUS_FILE : str
    all_schema : dict 

In [21]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml,create_directories

In [30]:
class ConfiguarationManager:
    def __init__(
        self,
        config_file = config_file_path,
        schema_file = schema_file_path,
        params_file = params_file_path):

        self.config = read_yaml(config_file)
        self.schema = read_yaml(schema_file)
        self.params = read_yaml(params_file)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE = config.STATUS_FILE,
            all_schema = schema
        )
        return data_validation_config


In [31]:
import os

from ml_project import logger


In [43]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema_key = self.config.all_schema.keys()
            all_schema = self.config.all_schema
            value  = self.config.all_schema.values()
            
            for col in all_cols:
                
                if col not in all_schema_key:
                    for i in all_schema:
                        print(data[col].dtypes ,'::',all_schema[i])
                        if all_schema[i] != data[col].dtypes:
                            validation_status = False
                            with open(self.config.STATUS_FILE, 'w') as f:
                                f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [44]:
try:
    config = ConfiguarationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e


[2025-04-15 11:22:31,678 : INFO : common : yaml file: config/config.yaml loaded successfully]
[2025-04-15 11:22:31,686 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-04-15 11:22:31,690 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-04-15 11:22:31,692 : INFO : common : created directory at: artifacts]
[2025-04-15 11:22:31,693 : INFO : common : created directory at: artifacts/data_validation]
